In [29]:
library(dplyr)

#Choose the type of TCGA dataset
cancer_type = "stomach_esophageal"

#You need to save all your datasets in this folder 
path <- paste0("/home/mxenoc/workspace/multiomics-benchmark/data/RData/", cancer_type, "/files/")

setwd(path)

In [30]:
dataset_names <- list.files(path)
dataset_better_names <- tools::file_path_sans_ext(dataset_names)

datasets = list()

#Prepare your dataset
for (i in 1:length(dataset_names)){
  
  dataset <- read.table(dataset_names[i], fill = TRUE)
  
  #Replace empty with NA 
  dataset <- mutate_all(dataset, list(~na_if(.,"")))
    
  #Omit NAs
  dataset <- na.omit(dataset)
  
  dataset <- dataset[!duplicated(dataset[,1]),]
  rownames(dataset) <- as.character(unlist(dataset[,1]))
  dataset <- dataset[,-1]
  
  dataset <- data.frame(t(dataset))
  
  colnames(dataset) <- c("Patients", paste0(dataset_better_names[i], 1:(ncol(dataset)-1)))

  datasets[[i]] <- dataset
}

names(datasets) <- dataset_better_names

#Merge them together 
all = Reduce(function(x, y) merge(x, y, by = "Patients", all = FALSE), datasets)

save(all, file = paste0('/home/mxenoc/workspace/multiomics-benchmark/data/RData/', cancer_type, "/", cancer_type, ".RData"))